-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

<i18n value="405115f0-8ffe-4a64-861b-b4a6772972be"/>



# End-to-End ETL in the Lakehouse
## Final Steps

We are picking up from the first notebook in this lab, [DE 12.2.1L - Instructions and Configuration]($./DE 12.2.1L - Instructions and Configuration)

If everything is setup correctly, you should have:
* A DLT Pipeline running in **Continuous** mode
* A job that is feeding that pipeline new data every 2 minutes
* A series of Databricks SQL Queries analysing the outputs of that pipeline

In [0]:
%run ../../Includes/Classroom-Setup-12.2.4L

Python interpreter will be restarted.
Python interpreter will be restarted.



Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02"

Validating the locally installed datasets:
| listing local files...(8 seconds)
| completed (8 seconds total)

Using the "default" schema.

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/cap_12
| DA.paths.user_db:     dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/cap_12/database.db
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02
| DA.paths.checkpoints: dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/cap_12/_checkpoints

Setup completed (9 seconds)


<i18n value="ce3151f0-a3dc-4cd5-9f92-a075c851101a"/>


## Execute a Query to Repair Broken Data

Review the code that defined the **`recordings_enriched`** table to identify the filter applied for the quality check.

In the cell below, write a query that returns all the records from the **`recordings_bronze`** table that were refused by this quality check.

In [0]:
%sql
-- ANSWER
SELECT * FROM ${da.schema_name}.recordings_bronze WHERE heartrate <= 0

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-4094000743659801> in <cell line: 1>()
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

<command-4094000743659801> in ____databricks_percent_sql()
      2   def ____databricks_percent_sql():
      3     import base64
----> 4     df = spark.sql(base64.standard_b64decode("U0VMRUNUICogRlJPTSAke2RhLnNjaGVtYV9uYW1lfS5yZWNvcmRpbmdzX2Jyb256ZSBXSEVSRSBoZWFydHJhdGUgPD0gMA==").decode())
      5     display(df)
      6     return df

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, f

<i18n value="25e86690-7fc4-45ea-b567-c6a229cef220"/>


For the purposes of our demo, let's assume that thorough manual review of our data and systems has demonstrated that occasionally otherwise valid heartrate recordings are returned as negative values.

Run the following query to examine these same rows with the negative sign removed.

In [0]:
%sql
SELECT abs(heartrate), * FROM ${da.schema_name}.recordings_bronze WHERE heartrate <= 0

<i18n value="6f853894-27ff-4edb-8e64-ff2fa3c0be39"/>


To complete our dataset, we wish to insert these fixed records into the silver **`recordings_enriched`** table.

Use the cell below to update the query used in the DLT pipeline to execute this repair.

**NOTE**: Make sure you update the code to only process those records that were previously rejected due to the quality check.

In [0]:
%sql
-- ANSWER

MERGE INTO ${da.schema_name}.recordings_enriched t
USING (SELECT
  CAST(a.device_id AS INTEGER) device_id, 
  CAST(a.mrn AS LONG) mrn, 
  abs(CAST(a.heartrate AS DOUBLE)) heartrate, 
  CAST(from_unixtime(a.time, 'yyyy-MM-dd HH:mm:ss') AS TIMESTAMP) time,
  b.name
  FROM ${da.schema_name}.recordings_bronze a
  INNER JOIN ${da.schema_name}.pii b
  ON a.mrn = b.mrn
  WHERE heartrate <= 0) v
ON t.mrn=v.mrn AND t.time=v.time
WHEN NOT MATCHED THEN INSERT *

<i18n value="03ebb8bd-781d-4d5d-87ff-f4a842e2419a"/>


Use the cell below to manually or programmatically confirm that this update has been successful.

(The total number of records in the **`recordings_bronze`** should now be equal to the total records in **`recordings_enriched`**).

In [0]:
# ANSWER
assert spark.table(f"{DA.schema_name}.recordings_bronze").count() == spark.table(f"{DA.schema_name}.recordings_enriched").count()

<i18n value="9b9685f0-dc1a-490c-92e9-ec068c7064d5"/>


## Consider Production Data Permissions

Note that while our manual repair of the data was successful, as the owner of these datasets, by default we have permissions to modify or delete these data from any location we're executing code.

To put this another way: our current permissions would allow us to change or drop our production tables permanently if an errant SQL query is accidentally executed with the current user's permissions (or if other users are granted similar permissions).

While for the purposes of this lab, we desired to have full permissions on our data, as we move code from development to production, it is safer to leverage <a href="https://docs.databricks.com/administration-guide/users-groups/service-principals.html" target="_blank">service principals</a> when scheduling Jobs and DLT Pipelines to avoid accidental data modifications.

<i18n value="15d7d859-5788-4c66-88f8-f276dba39347"/>


## Shut Down Production Infrastructure

Note that Databricks Jobs, DLT Pipelines, and scheduled DBSQL queries and dashboards are all designed to provide sustained execution of production code. In this end-to-end demo, you were instructed to configure a Job and Pipeline for continuous data processing. To prevent these workloads from continuing to execute, you should **Pause** your Databricks Job and **Stop** your DLT pipeline. Deleting these assets will also ensure that production infrastructure is terminated.

**NOTE**: All instructions for DBSQL asset scheduling in previous lessons instructed users to set the update schedule to end tomorrow. You may choose to go back and also cancel these updates to prevent Databricks SQL warehouses from staying on until that time.

In [0]:
DA.cleanup()

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>